**Задание 1**
Используем файл keywords.csv.
Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону.
Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона.
Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [60]:
#Функция определения региона по ключевому слову в запросе
#В случае если ниодного города из списка не найдено возвращает undefined
def geo_data(row):
    if 'москва' in row['keyword'] or 'тула' in row['keyword'] or 'ярославль' in row['keyword']:
        return 'Центр'
    elif 'петербург' in row['keyword'] or 'псков' in row['keyword'] or 'мурманск' in row['keyword']:
        return 'Северо-Запад'
    elif 'владивосток' in row['keyword'] or 'сахалин' in row['keyword'] or 'хабаровск' in row['keyword']:
        return 'Дальний Восток' 
    else:
        return 'undefined'

In [61]:
import pandas as pd
data = pd.read_csv('keywords.csv')
data['region']=data.apply(geo_data, axis=1) # применение функции определения региона
data[data['region']!='undefined'].head() # вывод первых записей в которых опредлен регион

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [28]:
#Функция классификации фильмов
def film(row):
    if row['rating']<=2.0:
        return 'низкий рейтинг'
    elif row['rating']<=4.0:
        return 'средний рейтинг'
    elif row['rating']<=5.0:
        return 'высокий рейтинг' 
    else:
        return 'undefined'

In [29]:
import pandas as pd
ratings = pd.read_csv('ratings.csv')
ratings['class']=ratings.apply(film, axis=1)
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,средний рейтинг
1,1,3,4.0,964981247,средний рейтинг
2,1,6,4.0,964982224,средний рейтинг
3,1,47,5.0,964983815,высокий рейтинг
4,1,50,5.0,964982931,высокий рейтинг


**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов).
Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [16]:
import pandas as pd
#Определяем первую таблицу, в которой определим пользователей
#с количеством оценок больше 100
coun = pd.read_csv('ratings.csv')
coun=coun['userId'].value_counts().reset_index(name='count')
coun.columns = ['userId', 'count']
coun=coun[coun['count']>100]
coun
#Вторая таблица, групперуем по пользователям 
#Определяем минимальное время
ratings = pd.read_csv('ratings.csv')
mintime=ratings.groupby('userId').min()['timestamp'].reset_index(name='min')
mintime.columns = ['userId', 'min']
mintime
#Третья таблица, групперуем по пользователям 
#Определяем максимальное время время
ratings = pd.read_csv('ratings.csv')
maxtime= ratings.groupby('userId').max()['timestamp'].reset_index(name='max')
maxtime.columns = ['userId', 'max']
maxtime
#Последний этап, объединяем три таблицы используя функцию inner join
#И вычисляем 4 столбец, как разница между максимумом и минимум
coun = coun.merge(mintime, on='userId', how='inner')
coun = coun.merge(maxtime, on='userId', how='inner')
coun['Lifetime']= coun['max']-coun['min']
coun.head()

,userId,count,min,max,Lifetime
0,414,2698,961436216,1527978072,566541856
1,599,2478,1498456035,1519558046,21102011
2,474,2108,974666330,1238350606,263684276
3,448,1864,1019123866,1499620709,480496843
4,274,1346,1171172369,1318410436,147238067


**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас".
 Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия.
 Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [141]:
#Функция заполнения списка годами
def years(list_):
    it = 1950
    list_=[]
    while it<=2020:
        list_.append(str(it)) 
        it+=1
    return list_

In [142]:
#Функция определения года выпуска фильма
#В случае если не найдено возвращает undefined
def production_year(row):
    ye=[]
    ye = years(ye)
    movies=''
    for el in ye:
        if el in row['title']:
            movies=el
    if movies !='':
        return movies
    else:
        return '1900'

In [144]:
#%%time
import pandas as pd
rating = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
rating = rating.merge(movies, on='movieId', how='inner')
rating['year']=rating.apply(production_year,axis=1)
rating.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995


In [147]:
rat = rating.groupby('year').mean()['rating'].reset_index(name='mean')
rat.sort_values('mean', ascending=False).head()

,year,mean
8,1957,4.039535
5,1954,4.009191
13,1962,3.969466
3,1952,3.953125
15,1964,3.940160
